# Evaluate Decoding Time Distribution

This notebook will evaluate the decoding time distribution given a specific code, noise model and list of decoders.
We ensure that all the decoders will share the same sequence of syndrome for a fair comparison even at very small number of samples.

To execute this notebook with a custom code, noise and decoder, use
```sh
# srun --time=1-00:00:00 --mem=10G --cpus-per-task=2 \
python3 -m qec_lego_bench notebook-time-distribution ./dist/time_distribution_example.ipynb 'rsc(d=3,p=0.01)' --decoder 'mwpf(c=0)' --target-precision 0.1 --local-maximum-jobs 4
```

In [ ]:
code: str = "rsc(d@3,p@0.01)"
noise: str = "none"
decoders: list[str] = ["mwpf", "huf", "bposd(max_iter=5,ms_scaling_factor=0.5)"]

max_cpu_hours: float | None = None
target_precision: float = 0.04  # about 4000 errors for the configuration with the smallest 

slurm_maximum_jobs: int = 50  # start with a smaller number of workers to avoid resource waste
slurm_cores_per_node: int = 10  # (slurm_maximum_jobs // slurm_cores_per_node) should not exceed 200
slurm_mem_per_job: int = 4  # 4GB per job
slurm_extra: dict = dict(
    walltime = "1-00:00:00",  # adaptively shutdown if no more jobs
    queue = "scavenge",  # use with caution: dask does not seem to handle scavenge workers well
    job_extra_directives = ["--requeue"],  # use with scavenge partition will help spawn scavenged jobs
)

import multiprocessing
local_maximum_jobs: int = multiprocessing.cpu_count()

json_filename: str | None = None
cpu: str = "unknown"
force_finished: bool = False  # only plot the figure and do not run experiments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
code = code.replace("@", "=")
noise = noise.replace("@", "=")
decoders = [decoder.replace("@", "=").replace(";", ",") for decoder in decoders]

from qec_lego_bench.notebooks.time_distribution import *

if json_filename is None:
    json_filename = default_json_filename(code=code, noise=noise)
print("saving results to:", json_filename)


### Define the Monte Carlo job function

In [ ]:
jobs = [MonteCarloJob(code=code, noise=noise)]

monte_carlo_function = TimeDistributionMonteCarloFunction(decoders=decoders)

if not force_finished:
    print(monte_carlo_function(10, code=code, noise=noise, verbose=True))

### Define the strategy to submit jobs

In [ ]:
precision_submitter = PrecisionSubmitter(
    time_limit=max_cpu_hours * 3600 if max_cpu_hours is not None else None,
    min_precision=None,
    target_precision=target_precision,
)

def submitter(executor: MonteCarloJobExecutor) -> list[tuple[MonteCarloJob, int]]:
    submit = precision_submitter(executor)
    return submit

## The rest of the notebook runs the evaluation

In [ ]:
config = MonteCarloExecutorConfig()
config.max_submitted_job = max(config.max_submitted_job, 3 * slurm_maximum_jobs)
executor = MonteCarloJobExecutor(
    monte_carlo_function,
    jobs,
    config=config,
    filename=json_filename,
    result_type=MultiDecoderDecodingTimeDistribution,
)

client_connector = SlurmClientConnector(
    slurm_maximum_jobs=slurm_maximum_jobs,
    slurm_cores_per_node=slurm_cores_per_node,
    slurm_mem_per_job=slurm_mem_per_job,
    slurm_extra=slurm_extra,
    local_maximum_jobs=local_maximum_jobs,
)

In [ ]:
import time  # add some sleep to let them work properly in VScode Jupyter notebook

time.sleep(0.2)
progress_plotter = JobProgressPlotter()
time.sleep(0.2)
compare_decoder_plotter = TimeDistributionPlotter()
time.sleep(0.2)
memory_plotter = MemoryUsagePlotter()


def callback(executor: MonteCarloJobExecutor):
    progress_plotter(executor)
    time.sleep(0.1)
    compare_decoder_plotter(executor)
    time.sleep(0.1)
    memory_plotter(executor)
    time.sleep(0.1)


executor.execute(
    client_connector=client_connector,
    submitter=submitter,
    loop_callback=callback,
    shutdown_cluster=True,
    force_finished=force_finished,
)